# Redes neuronales recurrentes

En el módulo anterior, hemos estado utilizando representaciones semánticas ricas del texto y un clasificador lineal simple sobre las incrustaciones. Lo que hace esta arquitectura es capturar el significado agregado de las palabras en una oración, pero no tiene en cuenta el **orden** de las palabras, ya que la operación de agregación sobre las incrustaciones elimina esta información del texto original. Debido a que estos modelos no pueden modelar el orden de las palabras, no pueden resolver tareas más complejas o ambiguas como la generación de texto o la respuesta a preguntas.

Para capturar el significado de una secuencia de texto, necesitamos usar otra arquitectura de red neuronal, llamada **red neuronal recurrente**, o RNN. En una RNN, pasamos nuestra oración a través de la red un símbolo a la vez, y la red produce un **estado**, que luego pasamos nuevamente a la red junto con el siguiente símbolo.

Dada la secuencia de tokens de entrada $X_0,\dots,X_n$, la RNN crea una secuencia de bloques de red neuronal y entrena esta secuencia de extremo a extremo utilizando retropropagación. Cada bloque de red toma un par $(X_i,S_i)$ como entrada y produce $S_{i+1}$ como resultado. El estado final $S_n$ o la salida $X_n$ se pasa a un clasificador lineal para producir el resultado. Todos los bloques de red comparten los mismos pesos y se entrenan de extremo a extremo utilizando una sola pasada de retropropagación.

Debido a que los vectores de estado $S_0,\dots,S_n$ se pasan a través de la red, esta es capaz de aprender las dependencias secuenciales entre palabras. Por ejemplo, cuando la palabra *no* aparece en algún lugar de la secuencia, puede aprender a negar ciertos elementos dentro del vector de estado, lo que resulta en una negación.

> Dado que los pesos de todos los bloques de RNN en la imagen son compartidos, la misma imagen puede representarse como un solo bloque (a la derecha) con un bucle de retroalimentación recurrente, que pasa el estado de salida de la red nuevamente a la entrada.

Veamos cómo las redes neuronales recurrentes pueden ayudarnos a clasificar nuestro conjunto de datos de noticias.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Clasificador RNN simple

En el caso de una RNN simple, cada unidad recurrente es una red lineal sencilla que toma un vector de entrada concatenado y un vector de estado, y produce un nuevo vector de estado. PyTorch representa esta unidad con la clase `RNNCell`, y una red de dichas celdas como una capa `RNN`.

Para definir un clasificador RNN, primero aplicaremos una capa de incrustación para reducir la dimensionalidad del vocabulario de entrada, y luego añadiremos una capa RNN encima:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Nota:** Aquí usamos una capa de embedding no entrenada por simplicidad, pero para obtener resultados aún mejores podemos usar una capa de embedding preentrenada con embeddings de Word2Vec o GloVe, como se describió en la unidad anterior. Para una mejor comprensión, podrías adaptar este código para trabajar con embeddings preentrenados.

En nuestro caso, utilizaremos un cargador de datos con padding, de modo que cada lote tendrá un número de secuencias rellenadas con la misma longitud. La capa RNN tomará la secuencia de tensores de embedding y producirá dos salidas:  
* $x$ es una secuencia de salidas de las celdas RNN en cada paso  
* $h$ es el estado oculto final para el último elemento de la secuencia  

Luego aplicamos un clasificador lineal completamente conectado para obtener el número de clases.

> **Nota:** Las RNN son bastante difíciles de entrenar, porque una vez que las celdas RNN se despliegan a lo largo de la longitud de la secuencia, el número resultante de capas involucradas en la retropropagación es bastante grande. Por lo tanto, necesitamos seleccionar una tasa de aprendizaje pequeña y entrenar la red en un conjunto de datos más grande para obtener buenos resultados. Esto puede tomar bastante tiempo, por lo que se recomienda usar GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Memoria a Largo y Corto Plazo (LSTM)

Uno de los principales problemas de las RNN clásicas es el llamado problema de los **gradientes que se desvanecen**. Debido a que las RNN se entrenan de extremo a extremo en una sola pasada de retropropagación, tienen dificultades para propagar el error a las primeras capas de la red, y por lo tanto, la red no puede aprender relaciones entre tokens distantes. Una de las formas de evitar este problema es introducir una **gestión explícita del estado** mediante el uso de los llamados **puertas**. Hay dos arquitecturas más conocidas de este tipo: **Memoria a Largo y Corto Plazo** (LSTM) y **Unidad de Relevo Controlada** (GRU).

![Imagen que muestra un ejemplo de una celda de memoria a largo y corto plazo](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

La red LSTM está organizada de una manera similar a las RNN, pero hay dos estados que se pasan de capa en capa: el estado actual $c$ y el vector oculto $h$. En cada unidad, el vector oculto $h_i$ se concatena con la entrada $x_i$, y juntos controlan lo que sucede con el estado $c$ a través de las **puertas**. Cada puerta es una red neuronal con activación sigmoide (salida en el rango $[0,1]$), que puede interpretarse como una máscara bit a bit cuando se multiplica por el vector de estado. Las puertas son las siguientes (de izquierda a derecha en la imagen anterior):
* **Puerta de olvido**: toma el vector oculto y determina qué componentes del vector $c$ necesitamos olvidar y cuáles pasar.
* **Puerta de entrada**: toma información de la entrada y del vector oculto, e inserta esa información en el estado.
* **Puerta de salida**: transforma el estado mediante una capa lineal con activación $\tanh$, y luego selecciona algunos de sus componentes usando el vector oculto $h_i$ para producir el nuevo estado $c_{i+1}$.

Los componentes del estado $c$ pueden interpretarse como banderas que se pueden activar o desactivar. Por ejemplo, cuando encontramos un nombre como *Alice* en la secuencia, podríamos asumir que se refiere a un personaje femenino y activar la bandera en el estado que indica que hay un sustantivo femenino en la oración. Más adelante, al encontrar frases como *and Tom*, activaríamos la bandera que indica que hay un sustantivo en plural. Así, manipulando el estado, supuestamente podemos hacer un seguimiento de las propiedades gramaticales de las partes de la oración.

> **Nota**: Un excelente recurso para entender los detalles internos de las LSTM es este gran artículo [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) de Christopher Olah.

Aunque la estructura interna de una celda LSTM puede parecer compleja, PyTorch oculta esta implementación dentro de la clase `LSTMCell` y proporciona el objeto `LSTM` para representar toda la capa LSTM. Por lo tanto, la implementación de un clasificador LSTM será bastante similar a la RNN simple que vimos anteriormente:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Secuencias empaquetadas

En nuestro ejemplo, tuvimos que rellenar todas las secuencias en el minibatch con vectores de ceros. Aunque esto genera cierto desperdicio de memoria, con las RNN es más crítico que se creen celdas adicionales para los elementos de entrada rellenados, las cuales participan en el entrenamiento pero no contienen información importante. Sería mucho mejor entrenar la RNN únicamente con el tamaño real de la secuencia.

Para lograr esto, se introduce un formato especial de almacenamiento de secuencias rellenadas en PyTorch. Supongamos que tenemos un minibatch rellenado que se ve así:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Aquí, 0 representa los valores rellenados, y el vector de longitud real de las secuencias de entrada es `[5,3,1]`.

Para entrenar eficazmente una RNN con secuencias rellenadas, queremos comenzar el entrenamiento del primer grupo de celdas de la RNN con un minibatch grande (`[1,6,9]`), pero luego terminar el procesamiento de la tercera secuencia y continuar el entrenamiento con minibatches más pequeños (`[2,7]`, `[3,8]`), y así sucesivamente. Por lo tanto, una secuencia empaquetada se representa como un solo vector - en nuestro caso `[1,6,9,2,7,3,8,4,5]`, y un vector de longitud (`[5,3,1]`), a partir del cual podemos reconstruir fácilmente el minibatch rellenado original.

Para generar una secuencia empaquetada, podemos usar la función `torch.nn.utils.rnn.pack_padded_sequence`. Todas las capas recurrentes, incluidas RNN, LSTM y GRU, admiten secuencias empaquetadas como entrada y producen una salida empaquetada, que puede ser decodificada usando `torch.nn.utils.rnn.pad_packed_sequence`.

Para poder generar una secuencia empaquetada, necesitamos pasar el vector de longitud a la red, y por lo tanto necesitamos una función diferente para preparar los minibatches:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

La red real sería muy similar a `LSTMClassifier` mencionado anteriormente, pero el paso `forward` recibirá tanto el minibatch con padding como el vector de longitudes de las secuencias. Después de calcular la incrustación, calculamos la secuencia empaquetada, la pasamos a la capa LSTM y luego desempaquetamos el resultado.

> **Nota**: En realidad no usamos el resultado desempaquetado `x`, porque utilizamos la salida de las capas ocultas en los cálculos posteriores. Por lo tanto, podemos eliminar el desempaquetado por completo de este código. La razón por la que lo colocamos aquí es para que puedas modificar este código fácilmente, en caso de que necesites usar la salida de la red en cálculos adicionales.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Nota:** Es posible que hayas notado el parámetro `use_pack_sequence` que pasamos a la función de entrenamiento. Actualmente, la función `pack_padded_sequence` requiere que el tensor de la secuencia de longitud esté en el dispositivo CPU, y por lo tanto, la función de entrenamiento necesita evitar mover los datos de la secuencia de longitud a la GPU durante el entrenamiento. Puedes revisar la implementación de la función `train_emb` en el archivo [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNNs bidireccionales y multicapa

En nuestros ejemplos, todas las redes recurrentes operaban en una sola dirección, desde el inicio de una secuencia hasta el final. Esto parece natural, ya que se asemeja a la forma en que leemos y escuchamos el habla. Sin embargo, dado que en muchos casos prácticos tenemos acceso aleatorio a la secuencia de entrada, podría tener sentido realizar cálculos recurrentes en ambas direcciones. Estas redes se llaman **RNNs bidireccionales**, y se pueden crear pasando el parámetro `bidirectional=True` al constructor de RNN/LSTM/GRU.

Al trabajar con una red bidireccional, necesitaríamos dos vectores de estado oculto, uno para cada dirección. PyTorch codifica esos vectores como un solo vector de tamaño doble, lo cual es bastante conveniente, porque normalmente pasarías el estado oculto resultante a una capa lineal completamente conectada, y solo tendrías que tener en cuenta este aumento de tamaño al crear la capa.

Una red recurrente, ya sea unidireccional o bidireccional, captura ciertos patrones dentro de una secuencia y puede almacenarlos en el vector de estado o pasarlos a la salida. Al igual que con las redes convolucionales, podemos construir otra capa recurrente encima de la primera para capturar patrones de nivel superior, construidos a partir de los patrones de bajo nivel extraídos por la primera capa. Esto nos lleva al concepto de **RNN multicapa**, que consiste en dos o más redes recurrentes, donde la salida de la capa anterior se pasa a la siguiente capa como entrada.

![Imagen que muestra una RNN multicapa de memoria a largo y corto plazo](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Imagen tomada de [este maravilloso artículo](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) por Fernando López*

PyTorch facilita la construcción de este tipo de redes, ya que solo necesitas pasar el parámetro `num_layers` al constructor de RNN/LSTM/GRU para construir automáticamente varias capas de recurrencia. Esto también significa que el tamaño del vector de estado oculto aumentará proporcionalmente, y deberás tener esto en cuenta al manejar la salida de las capas recurrentes.


## RNNs para otras tareas

En esta unidad, hemos visto que las RNNs pueden usarse para la clasificación de secuencias, pero de hecho, pueden manejar muchas más tareas, como la generación de texto, la traducción automática y más. Consideraremos esas tareas en la próxima unidad.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
